### Installating and importing h2o and boto3

In [1]:
!pip install h2o boto3

In [2]:
import boto3
from h2o.automl import H2OAutoML
import requests

In [3]:
SECRET="AKIAtEpiGWUcQIelPRlD1Pi6xQ"
SECRETACCESS= "YNV6xS8lXnCTGSy1x2vGkmGnmdJbZSapNXaSaRhK"
URL = "http://uvo1mtax5lmyplwknci.vm.cld.sr"

### Uploading a file to CORTX using Boto3 

This will allow H2O to download it via their s3 api later on


In [4]:
#Creating Session With Boto3.
session = boto3.session.Session()

s3_client = session.client(
aws_access_key_id=SECRET,
aws_secret_access_key=SECRETACCESS,
endpoint_url=URL,
use_ssl=False,service_name="s3"
)

# Getting the data and placing it in Seagate CORTX before downloading it
response = requests.get('https://github.com/h2oai/h2o-tutorials/raw/master/h2o-world-2017/automl/data/powerplant_output.csv')

s3_client.put_object(Body=response.text,Bucket="testbucket",Key="h2o.csv")

{'ETag': '"07b88846413d438e3460b846526a1025"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'content-type': 'text/plain',
   'etag': '"07b88846413d438e3460b846526a1025"',
   'server': 'SeagateS3',
   'x-amz-request-id': '85ff666c-84bd-42c4-b02e-5f99c4df19a5'},
  'HTTPStatusCode': 200,
  'HostId': '',
  'RequestId': '85ff666c-84bd-42c4-b02e-5f99c4df19a5',
  'RetryAttempts': 0}}

### Running H2O

Using data from CORTX Seagate

Specific arguments parsing in order to utilise Seagate CORTX endpoints - this is important and cannot be skipped

In [5]:
with open("core-site.xml","w") as f:
    f.write(f"""<property>
  <name>fs.s3.awsAccessKeyId</name>
  <value>{SECRET}</value>
</property>

<property>
  <name>fs.s3.awsSecretAccessKey</name>
  <value>{SECRETACCESS}</value>
</property>
""")
    
import h2o
h2o.init(jvm_custom_args=[f"-Dsys.ai.h2o.persist.s3.endPoint={URL}","-Dsys.ai.h2o.persist.s3.enable.path.style=true"],extra_classpath=["-hdfs_config core-site.xml"])

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp_2c68edw
  JVM stdout: /tmp/tmp_2c68edw/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp_2c68edw/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.4
H2O_cluster_version_age:,24 days
H2O_cluster_name:,H2O_from_python_unknownUser_jgk1bq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


Set s3 credentials for downloading from Seagate's CORTX

In [6]:
# Set s3 credentials
from h2o.persist import set_s3_credentials
set_s3_credentials(SECRET, SECRETACCESS)

Credentials successfully set.


In [7]:
# Load data into H2O from Seagate CORTX Server
df = h2o.import_file(path = "s3://testbucket/h2o.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


The data has been fully downloaded at this point here - accessed directly from CORTX server via the s3 layer ("s3://testbucket/h2o.csv")

The next few steps is utilising the data in H2O's AutoML pipelines to achieve State-of-the-art results on the dataset



### Running H2O's AutoML

Let's take a look at the data.

In [8]:
df.describe()

Rows:9568
Cols:5




,TemperatureCelcius,ExhaustVacuumHg,AmbientPressureMillibar,RelativeHumidity,HourlyEnergyOutputMW
type,real,real,real,real,real
mins,1.81,25.36,992.89,25.56,420.26
mean,19.651231187290957,54.3058037207358,1013.2590781772578,73.30897784280936,454.36500940635455
maxs,37.11,81.56,1033.3,100.16,495.76
sigma,7.452473229611082,12.707892998326807,5.93878370581162,14.600268756728957,17.066994999803423
zeros,0,0,0,0,0
missing,0,0,0,0,0
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.4,1012.16,92.14,488.56


Next, let's identify the response column and save the column name as `y`.  In this dataset, we will use all columns except the response as predictors, so we can skip setting the `x` argument explicitly.

In [9]:
y = "HourlyEnergyOutputMW"

Lastly, let's split the data into two frames, a `train` (80%) and a `test` frame (20%).  The `test` frame will be used to score the leaderboard and to demonstrate how to generate predictions using an AutoML leader model.

In [10]:
splits = df.split_frame(ratios = [0.8], seed = 1)
train = splits[0]
test = splits[1]

## Run AutoML 

Run AutoML, stopping after 60 seconds.  The `max_runtime_secs` argument provides a way to limit the AutoML run by time.  When using a time-limited stopping criterion, the number of models train will vary between runs.  If different hardware is used or even if the same machine is used but the available compute resources on that machine are not the same between runs, then AutoML may be able to train more models on one run vs another. 

The `test` frame is passed explicitly to the `leaderboard_frame` argument here, which means that instead of using cross-validated metrics, we use test set metrics for generating the leaderboard.

In [11]:
aml = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "powerplant_lb_frame")
aml.train(y = y, training_frame = train, leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


*Note: If you see the following error, it means that you need to install the pandas module.*
```
H2OTypeError: Argument `python_obj` should be a None | list | tuple | dict | numpy.ndarray | pandas.DataFrame | scipy.sparse.issparse, got H2OTwoDimTable 
``` 

For demonstration purposes, we will also execute a second AutoML run, this time providing the original, full dataset, `df` (without passing a `leaderboard_frame`).  This is a more efficient use of our data since we can use 100% of the data for training, rather than 80% like we did above.  This time our leaderboard will use cross-validated metrics.

*Note: Using an explicit `leaderboard_frame` for scoring may be useful in some cases, which is why the option is available.*  

In [12]:
aml2 = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "powerplant_full_data")
aml2.train(y = y, training_frame = df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


*Note: We specify a `project_name` here for clarity.*

## Leaderboard

Next, we will view the AutoML Leaderboard.  Since we specified a `leaderboard_frame` in the `H2OAutoML.train()` method for scoring and ranking the models, the AutoML leaderboard uses the performance on this data to rank the models.

After viewing the `"powerplant_lb_frame"` AutoML project leaderboard, we compare that to the leaderboard for the `"powerplant_full_data"` project.  We can see that the results are better when the full dataset is used for training.  

A default performance metric for each machine learning task (binary classification, multiclass classification, regression) is specified internally and the leaderboard will be sorted by that metric.  In the case of regression, the default ranking metric is mean residual deviance.  In the future, the user will be able to specify any of the H2O metrics so that different metrics can be used to generate rankings on the leaderboard.

In [13]:
aml.leaderboard.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_grid__1_AutoML_20210801_203326_model_1,11.7239,3.42402,11.7239,2.34031,0.00751901
XGBoost_grid__1_AutoML_20210801_203326_model_2,14.4687,3.80377,14.4687,2.75897,0.00835398
XGBoost_2_AutoML_20210801_203326,15.2316,3.90277,15.2316,2.88963,0.00857461
XGBoost_3_AutoML_20210801_203326,15.9585,3.99481,15.9585,3.01604,0.00879275
GBM_grid__1_AutoML_20210801_203326_model_1,15.9601,3.99501,15.9601,2.97285,0.00878345
GLM_1_AutoML_20210801_203326,21.0746,4.59071,21.0746,3.60825,0.0100994
XRT_1_AutoML_20210801_203326,21.4955,4.63633,21.4955,3.28516,0.0102065
DeepLearning_1_AutoML_20210801_203326,22.5645,4.75021,22.5645,3.5958,0.0104512
DeepLearning_grid__1_AutoML_20210801_203326_model_1,23.9207,4.89088,23.9207,3.7618,0.0107912
GBM_5_AutoML_20210801_203326,30.8561,5.55483,30.8561,4.47109,0.0121477


Now we will view a snapshot of the top models.  Here we should see the two Stacked Ensembles at or near the top of the leaderboard.  Stacked Ensembles can almost always outperform a single model.

In [14]:
aml2.leaderboard.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_grid__1_AutoML_20210801_203431_model_1,10.7323,3.27602,10.7323,2.30761,0.00719845
XGBoost_1_AutoML_20210801_203431,13.1501,3.62631,13.1501,2.70778,0.00796222
XGBoost_2_AutoML_20210801_203431,13.2317,3.63754,13.2317,2.70402,0.0079882
XGBoost_grid__1_AutoML_20210801_203431_model_2,13.5327,3.67868,13.5327,2.71647,0.00806812
XGBoost_3_AutoML_20210801_203431,14.6819,3.8317,14.6819,2.89557,0.00841354
GBM_grid__1_AutoML_20210801_203431_model_1,14.9633,3.86824,14.9633,2.92079,0.00849118
GBM_grid__1_AutoML_20210801_203431_model_2,16.0995,4.01242,16.0995,3.08456,0.00878243
DRF_1_AutoML_20210801_203431,16.6506,4.08052,16.6506,2.95398,0.00896094
XRT_1_AutoML_20210801_203431,17.5849,4.19344,17.5849,3.03052,0.00920459
DeepLearning_1_AutoML_20210801_203431,19.0227,4.36151,19.0227,3.3543,0.00958201


This dataset comes from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant) of machine learning datasets.  The data was used in a [publication](https://www.sciencedirect.com/science/article/pii/S0142061514000908) in the *International Journal of Electrical Power & Energy Systems* in 2014.  In the paper, the authors achieved a mean absolute error (MAE) of 2.818 and a Root Mean-Squared Error (RMSE) of 3.787 on their best model.  So, with H2O's AutoML, we've already beaten the state-of-the-art in just 60 seconds of compute time!

## Predict Using Leader Model

If you need to generate predictions on a test set, you can make predictions on the `"H2OAutoML"` object directly, or on the leader model object.

In [15]:
pred = aml.predict(test)
pred.head()

xgboost prediction progress: |████████████████████████████████████████████| 100%


predict
485.695
473.756
467.413
450.41
448.128
468.133
442.936
466.101
442.655
431.161


If needed, the standard `model_performance()` method can be applied to the AutoML leader model and a test set to generate an H2O model performance object.

In [16]:
perf = aml.leader.model_performance(test)
perf


ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 11.723888817434862
RMSE: 3.4240164744689623
MAE: 2.3403112773602754
RMSLE: 0.007519009129557612
Mean Residual Deviance: 11.723888817434862
